In [6]:
from time_lstm import T1TimeLSTM, T2TimeLSTM
from dataset_delay_predictions import Dataset_Delay_Prediction, Dataset_Delay_Prediction_from_list, DatasetDelayPredictionStackOverflow
from time_dependant_representation import TimeDepMasking, TimeDepJointEmbedding
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Masking, Layer, LSTM
import os

In [7]:
import tensorflow as tf

In [10]:
import pickle

In [11]:
dataset_names = [
    '2019-10_11_12-dataset_delay_prediction',
    'stack-overflow-dataset'
]

In [63]:
dataset_name = dataset_names[0]
pickle_filename = dataset_name.replace('/', '-')
if os.path.exists(pickle_filename):
    print('Reading pickle file...')
    dataset = pickle.load(open(pickle_filename, 'rb'))

Reading pickle file...


In [42]:
training_set_length = dataset.training_set_length
X_train = dataset.full_features_dt[dataset.training_set_length:]
seqlen = dataset.full_seqlen[dataset.training_set_length:]
y_train = np.array(dataset.full_values[dataset.training_set_length:])
print(y_train.shape)

(5307, 1)


In [68]:
training_set_length = int(1/5 * len(dataset.full_features))
X_train = dataset.full_features[training_set_length:]
seqlen = dataset.full_seqlen[training_set_length:]
y_train = np.array(dataset.full_values[training_set_length:])
print(y_train.shape)

(4375, 1)


In [69]:
number_of_event = dataset.number_of_events
X_train_bis = []
number_of_event = len(X_train[0][0])
for idx in range(len(X_train)):
    seq = X_train[idx]
    ts_list = [[a[-1]] for a in seq]
    X_train_bis.append(ts_list)

padding_value = 0.123456789
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_train_bis,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')

In [70]:
print(padded_inputs.shape)
lstm_units = 50

(4375, 410, 1)


In [71]:
regressor = Sequential()
regressor.add(Masking(mask_value=padding_value))
regressor.add(LSTM(units=lstm_units))
regressor.add(Dense(units=y_train.shape[1], activation='sigmoid'))

In [72]:
number_of_epochs = 5

In [73]:

regressor.compile(optimizer='adam', loss='mean_squared_error')
history = regressor.fit(padded_inputs, y_train, batch_size=50, epochs=number_of_epochs, verbose=2)

Train on 4375 samples
Epoch 1/5


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f3b9e3fb730>
Traceback (most recent call last):
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/data/ops/iterator_ops.py", line 541, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/ops/gen_dataset_ops.py", line 1157, in delete_iterator
    "DeleteIterator", handle=handle, deleter=deleter, name=name)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py", line 793, in _apply_op_helper
    op_def=op_def)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/framework/func_graph.py", line 544, in create_op
    inp = self.capture(inp)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/ops/while_v2.py", line 919, in capt

4375/4375 - 33s - loss: 0.0268
Epoch 2/5
4375/4375 - 12s - loss: 0.0241
Epoch 3/5
4375/4375 - 12s - loss: 0.0243
Epoch 4/5
4375/4375 - 13s - loss: 0.0237
Epoch 5/5
4375/4375 - 12s - loss: 0.0234


In [76]:
X_test = dataset.full_features[:training_set_length]
seqlen_test = dataset.full_seqlen[:training_set_length]
y_test = np.array(dataset.full_values[:training_set_length])
X_test_bis = []
number_of_event = len(X_train[0][0])
for idx in range(training_set_length):
    seq = X_test[idx]
    ts_list = [[a[-1]] for a in seq]
    X_test_bis.append(ts_list)


In [79]:
padding_value = 0.123456789
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_test_bis,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')
pred = regressor.predict(padded_inputs)
mean_sum_of_squares = sum([(y_test[i] - pred[i])**2 for i in range(training_set_length)])/training_set_length
print(mean_sum_of_squares)

[0.02284944]


## LSTM Multi

In [52]:
padding_value = 0.123456789
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')

print(padded_inputs.shape)
regressor = Sequential()
regressor.add(Masking(mask_value=padding_value))
regressor.add(LSTM(units=lstm_units))
regressor.add(Dense(units=y_train.shape[1], activation='sigmoid'))


regressor.compile(optimizer='adam', loss='mean_squared_error')
history = regressor.fit(padded_inputs, y_train, batch_size=50, epochs=number_of_epochs, verbose=2)

(5307, 735, 23)
Train on 5307 samples
Epoch 1/5
5307/5307 - 35s - loss: 0.2294
Epoch 2/5
5307/5307 - 28s - loss: 0.2290
Epoch 3/5
5307/5307 - 29s - loss: 0.2290
Epoch 4/5
5307/5307 - 30s - loss: 0.2297
Epoch 5/5
5307/5307 - 30s - loss: 0.2288


In [54]:
X_test = dataset.full_features_dt[:dataset.training_set_length]
seqlen_test = dataset.full_seqlen[:dataset.training_set_length]
y_test = np.array(dataset.full_values[:dataset.training_set_length])

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')
pred = regressor.predict(padded_inputs)
mean_sum_of_squares = sum([(y_test[i] - pred[i])**2 for i in range(dataset.training_set_length)])/dataset.training_set_length
print(mean_sum_of_squares)

[0.22875992]


## T1TimeLSTM

In [62]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')
print(padded_inputs.shape)
regressor = Sequential()
regressor.add(Masking(mask_value=padding_value))
regressor.add(T1TimeLSTM(units=lstm_units))
regressor.add(Dense(units=y_train.shape[1], activation='sigmoid'))

regressor.compile(optimizer='adam', loss='mean_squared_error')
history = regressor.fit(padded_inputs, y_train, batch_size=1, epochs=number_of_epochs, verbose=2)


(5307, 735, 23)
Train on 5307 samples
Epoch 1/5


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/charles/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-62-39502405b399>", line 12, in <module>
    history = regressor.fit(padded_inputs, y_train, batch_size=1, epochs=number_of_epochs, verbose=2)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 728, in fit
    use_multiprocessing=use_multiprocessing)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 324, in fit
    total_epochs=epochs)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 123, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "/home/charles/anaconda3/envs/tf-2.0/lib/python3.7/site-packages/tensorflow

KeyboardInterrupt: 

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')
pred = regressor.predict(padded_inputs)
mean_sum_of_squares = sum([(y_test[i] - pred[i])**2 for i in range(dataset.training_set_length)])/dataset.training_set_length
print(mean_sum_of_squares)

## T2TimeLSTM

In [ ]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                                      padding='post',
                                                                      value=padding_value,
                                                                      dtype='float32')
print(padded_inputs.shape)
regressor = Sequential()
regressor.add(Masking(mask_value=padding_value))
regressor.add(T2TimeLSTM(units=lstm_units))
regressor.add(Dense(units=y_train.shape[1], activation='sigmoid'))

regressor.compile(optimizer='adam', loss='mean_squared_error')
history = regressor.fit(padded_inputs, y_train, batch_size=1, epochs=number_of_epochs, verbose=2)


In [ ]:
tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')
pred = regressor.predict(padded_inputs)
mean_sum_of_squares = sum([(y_test[i] - pred[i])**2 for i in range(dataset.training_set_length)])/dataset.training_set_length
print(mean_sum_of_squares)

## TimeDepJointEmbedding

In [ ]:
lstm_units = dataset.number_of_event
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')

print(padded_inputs.shape)
regressor = Sequential()
regressor.add(Masking(mask_value=padding_value))
regressor.add(TimeDepJointEmbedding(units=lstm_units))
regressor.add(Dense(units=y_train.shape[1], activation='sigmoid'))

regressor.compile(optimizer='adam', loss='mean_squared_error')
history = regressor.fit(padded_inputs, y_train, batch_size=1, epochs=number_of_epochs, verbose=2)
X_test = np.array([padded_inputs[0, :, :]])

In [ ]:
tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                              padding='post',
                                                              value=padding_value,
                                                              dtype='float32')
pred = regressor.predict(padded_inputs)
mean_sum_of_squares = sum([(y_test[i] - pred[i])**2 for i in range(dataset.training_set_length)])/dataset.training_set_length
print(mean_sum_of_squares)